In [6]:
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
# import algorithms.mm_utils
# from FIS3 import FIS3
# from function_util import *
from datetime import datetime

In [8]:
gdf = pd.read_pickle('envirocar-tmp-64be1e21c7e42807e7956010.pkl')
print(gdf)

                           id                      time  \
0    64be1e21c7e42807e7956012 2023-07-24 06:45:51+00:00   
1    64be1e35c7e42807e7956018 2023-07-24 06:45:51+00:00   
2    64be1e35c7e42807e795601a 2023-07-24 06:45:53+00:00   
3    64be1e35c7e42807e795601b 2023-07-24 06:45:55+00:00   
4    64be1e35c7e42807e795601c 2023-07-24 06:45:57+00:00   
..                        ...                       ...   
568  64be22b4c7e42807e79563ea 2023-07-24 07:05:15+00:00   
569  64be22b4c7e42807e79563eb 2023-07-24 07:05:18+00:00   
570  64be22b4c7e42807e79563ec 2023-07-24 07:05:20+00:00   
571  64be22b4c7e42807e79563ed 2023-07-24 07:05:22+00:00   
572  64be22b4c7e42807e79563ee 2023-07-24 07:05:24+00:00   

                                           phenomenons  \
0    {'GPS Speed': {'value': 13.93499879611133, 'un...   
1    {'GPS Speed': {'value': 13.93499879611133, 'un...   
2    {'Consumption (GPS-based)': {'value': 0.796628...   
3    {'GPS Bearing': {'value': 132.66633441084105, ...   
4

In [10]:


# phenomenons contains trajectory information of each time point in dictionary format
# example of extracting phenomenons from first data point
dict = gdf['phenomenons'][0]

# create array that save trajectory information that we want to extract 
key_list = ['GPS Speed', 'GPS HDOP', 'GPS Bearing']

# Initialize data frame 
df = pd.DataFrame()

# loop to get trajectory infos for each point 
for key in key_list:
    temp = []
    for i in range(gdf.shape[0]):
        # print(i)
        dict_temp = gdf['phenomenons'][i]
        if key in dict_temp:
            temp.append(dict_temp[key]['value'])
        else:
            temp.append(float("nan"))
    df[key] = temp


# combine geodata frame with the new extracted info    
gdf = pd.concat([gdf, df],axis = 1)



In [11]:
gdf

,id,time,phenomenons,geometry,lat_lon,GPS Speed,GPS HDOP,GPS Bearing,GPS Speed,GPS HDOP,GPS Bearing
0,64be1e21c7e42807e7956012,2023-07-24 06:45:51+00:00,"{'GPS Speed': {'value': 13.93499879611133, 'un...",POINT (511368.642 5405689.381),POINT (9.15483 48.80409),13.934999,1.448255,182.221056,13.934999,1.448255,182.221056
1,64be1e35c7e42807e7956018,2023-07-24 06:45:51+00:00,"{'GPS Speed': {'value': 13.93499879611133, 'un...",POINT (511368.642 5405689.381),POINT (9.15483 48.80409),13.934999,1.448255,182.221056,13.934999,1.448255,182.221056
2,64be1e35c7e42807e795601a,2023-07-24 06:45:53+00:00,{'Consumption (GPS-based)': {'value': 0.796628...,POINT (511373.377 5405681.773),POINT (9.15489 48.80402),14.755801,1.250642,138.580838,14.755801,1.250642,138.580838
3,64be1e35c7e42807e795601b,2023-07-24 06:45:55+00:00,"{'GPS Bearing': {'value': 132.66633441084105, ...",POINT (511379.964 5405679.397),POINT (9.15498 48.80400),6.671649,1.097581,132.666334,6.671649,1.097581,132.666334
4,64be1e35c7e42807e795601c,2023-07-24 06:45:57+00:00,"{'GPS PDOP': {'value': 1.6498814225196838, 'un...",POINT (511385.445 5405676.830),POINT (9.15505 48.80398),12.546036,0.900000,153.440685,12.546036,0.900000,153.440685
...,...,...,...,...,...,...,...,...,...,...,...
568,64be22b4c7e42807e79563ea,2023-07-24 07:05:15+00:00,"{'GPS Altitude': {'value': 326.90938248122785,...",POINT (512405.100 5402498.651),POINT (9.16884 48.77537),22.812011,0.900000,225.568249,22.812011,0.900000,225.568249
569,64be22b4c7e42807e79563eb,2023-07-24 07:05:18+00:00,"{'GPS Accuracy': {'value': 3.5182861983776093,...",POINT (512393.907 5402497.908),POINT (9.16869 48.77536),16.407745,0.900000,287.462846,16.407745,0.900000,287.462846
570,64be22b4c7e42807e79563ec,2023-07-24 07:05:20+00:00,"{'GPS VDOP': {'value': 1.3492551773786543, 'un...",POINT (512388.408 5402501.481),POINT (9.16862 48.77539),10.657951,0.849752,313.434778,10.657951,0.849752,313.434778
571,64be22b4c7e42807e79563ed,2023-07-24 07:05:22+00:00,{'Consumption (GPS-based)': {'value': 1.542616...,POINT (512387.141 5402503.279),POINT (9.16860 48.77541),2.728976,0.800000,313.575841,2.728976,0.800000,313.575841


In [29]:
import preprocessing.constants as cst

def conc(a):
    #function to convert list or integer in osmid into a unique string id 
    if type(a) is int:
        return str(a)
    ans = ",".join(map(str, a))
    return ans


# print(gdf.crs)
# get road network
# Get the bounding box
bbox = gdf.to_crs(cst.EPSG4326).total_bounds# 'total_bounds' returns a tuple with (minx, miny, maxx, maxy) values
minx, miny, maxx, maxy = bbox# Download a map by specifying the bounding box
# and draw the graph
G = ox.graph.graph_from_bbox(maxy, miny, maxx, minx, network_type='drive')
graph_proj = ox.project_graph(G)
nodes_utm, edges_utm = ox.graph_to_gdfs(graph_proj, nodes=True, edges=True)# extract road info
nodes, edges = ox.graph_to_gdfs(G)# append latitude and longitude to utm edges
edges_utm['lon_lat'] = edges['geometry']# convert osmid into unique string id
edges_utm['str_id'] = edges_utm['osmid'].apply(conc)# append latitude and longitude to utm edges
nodes_utm['lon_lat'] = nodes['geometry']

In [31]:
nodes_utm

,y,x,street_count,lon,lat,highway,geometry,lon_lat
osmid,,,,,,,,
490694,5.401964e+06,511612.522924,4,9.158041,48.770573,NaN,POINT (511612.523 5401964.048),POINT (9.15804 48.77057)
645239,5.402013e+06,511738.098122,4,9.159752,48.771008,NaN,POINT (511738.098 5402012.620),POINT (9.15975 48.77101)
643986,5.401916e+06,511495.828214,4,9.156452,48.770142,NaN,POINT (511495.828 5401915.930),POINT (9.15645 48.77014)
640759,5.402065e+06,511565.868777,4,9.157409,48.771482,traffic_signals,POINT (511565.869 5402065.008),POINT (9.15741 48.77148)
490704,5.402056e+06,511848.524107,4,9.161256,48.771395,NaN,POINT (511848.524 5402055.883),POINT (9.16126 48.77139)
...,...,...,...,...,...,...,...,...
2853072225,5.402257e+06,512323.874178,3,9.167731,48.773195,NaN,POINT (512323.874 5402257.011),POINT (9.16773 48.77320)
2942602369,5.402099e+06,510106.875409,1,9.137553,48.771809,NaN,POINT (510106.875 5402098.522),POINT (9.13755 48.77181)
3086420228,5.402006e+06,510603.235246,3,9.144306,48.770973,NaN,POINT (510603.235 5402006.476),POINT (9.14431 48.77097)


In [32]:
edges_utm

osmid      lanes  \
u          v          key                                                 
490694     645239     0                [73855721, 761494487]          3   
           643986     0    [281015264, 235147317, 235147318]  [4, 5, 3]   
           640759     0     [73855929, 581282086, 235147319]     [2, 3]   
645239     645212     0                             51690051        NaN   
           490704     0                             93853485          3   
...                                                      ...        ...   
8808586013 8806255239 0                             25409555          2   
           8806255240 0                           1017445581          3   
           9469563482 0                           1027127715        NaN   
9384547814 18240902   0                             10319573          1   
           9384547816 0                           1017448237        NaN   

                              ref            name      highway maxspeed  \
u          v          key                                                 
490694     645239     0    L 1015  Rotebühlstraße    secondary       40   
           643986     0    L 1015  Rotebühlstraße    secondary       40   
           640759     0       NaN    Schwabstraße     tertiary       40   
645239     645212     0       NaN  Reuchlinstraße  residential       30   
           490704     0    L 1015  Rotebühlstraße    secondary       40   
...                           ...             ...          ...      ...   
8808586013 8806255239 0    L 1187   Am Kräherwald    secondary       40   
           8806255240 0    L 1187   Am Kräherwald    secondary       40   
           9469563482 0       NaN       Lenzhalde     tertiary      NaN   
9384547814 18240902   0       NaN       Lenzhalde     tertiary       40   
           9384547816 0       NaN             NaN     tertiary      NaN   

                           oneway reversed   length  \
u          v          key                             
490694     645239     0     False    False  134.348   
           643986     0     False     True  125.956   
           640759     0     False    False  111.198   
645239     645212     0      True    False  154.434   
           490704     0     False    False  118.340   
...                           ...      ...      ...   
8808586013 8806255239 0     False     True   27.374   
           8806255240 0     False    False   17.383   
           9469563482 0      True    False   18.237   
9384547814 18240902   0      True    False   12.121   
           9384547816 0      True    False   43.128   

                                                                    geometry  \
u          v          key                                                      
490694     645239     0    LINESTRING (511612.523 5401964.048, 511622.119...   
           643986     0    LINESTRING (511612.523 5401964.048, 511596.886...   
           640759     0    LINESTRING (511612.523 5401964.048, 511608.508...   
645239     645212     0    LINESTRING (511738.098 5402012.620, 511742.062...   
           490704     0    LINESTRING (511738.098 5402012.620, 511776.585...   
...                                                                      ...   
8808586013 8806255239 0    LINESTRING (511090.869 5403823.008, 511079.172...   
           8806255240 0    LINESTRING (511090.869 5403823.008, 511097.323...   
           9469563482 0    LINESTRING (511090.869 5403823.008, 511090.996...   
9384547814 18240902   0    LINESTRING (511121.321 5403819.845, 511131.202...   
           9384547816 0    LINESTRING (511121.321 5403819.845, 511139.394...   

                                 width bridge est_width access junction  \
u          v          key                                                 
490694     645239     0            NaN    NaN       NaN    NaN      NaN   
           643986     0            NaN    NaN       NaN    NaN      NaN   
           640759     0    [8.5, 13.4]    NaN 

In [ ]:
# append latitude and longitude to utm edges
# gdf_utm['lon_lat'] = gdf['geometry']# convert time str to datetime
gdf_utm['time'] = pd.to_datetime(gdf_utm['time'])
gdf_utm['speed_mps'] = gdf_utm['GPS Speed']/3.6